# Automated ML (Titanic Survival Prediction Project)

Importing Dependencies needed to to complete the project:

In [2]:
#Import libraries needed
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from azureml.data.dataset_factory import TabularDatasetFactory
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [3]:
#Testing the authentication using the Workspace method "from_config"
Workspace.from_config()

Workspace.create(name='quick-starts-ws-138164', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-138164')

In [4]:
#create workspace
ws = Workspace.from_config()

In [5]:
# Create Experiment

experiment_name = 'automl-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment-1,quick-starts-ws-138164,Link to Azure Machine Learning studio,Link to Documentation


## Dataset Overview:

As you probably have guessed from the project title we will be working with the "Titanic Dataset" which is already a classical dataset to learn Machine Learning.

The main task for this project will be to build a predictive model that answers the question: “what sorts of people were more likely to survive?” To answer the above stated question we are going to give the model different input variables such as age, type of cabin the passanger had, etc

In [6]:
#Getting the dataset
key = "titanic_dataset"
description_text = "Titanic Dataset for Capstone Project"

dataset = TabularDatasetFactory.from_delimited_files('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

#Register Dataset
dataset = dataset.register(workspace=ws,
                           name=key,
                           description=description_text)

#Create a df out of the registered dataset
dataset = dataset.to_pandas_dataframe()
dataset.describe()

,pclass,survived,sibsp,parch,fare
count,1309.000000,1309.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,0.498854,0.385027,33.295479
std,0.837836,0.486055,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,8.000000,9.000000,512.329200


In [7]:
###Be sure to have the compute target setup
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook138164"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.

Running


### Clean Data:

In [8]:
#Import your clean data function from the train.py file
from train import clean_data

In [9]:
#apply the function
x, y = clean_data(dataset)

In [10]:
#Scale the features
from sklearn.preprocessing import StandardScaler

In [11]:
# create scaler
variables = x.columns.tolist()

scaler = StandardScaler()
scaler.fit(x[variables]) 

x = scaler.transform(x[variables])

In [12]:
x = pd.DataFrame(x,columns=variables)


In [13]:
#Split the data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size = 0.2,
                                                    random_state=0)

In [14]:
#bring them together them again
dataset = pd.concat([x_train,y_train],axis=1)

In [15]:
dataset.head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
1118,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
44,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
1072,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
1130,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
574,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


In [16]:
#To train the model we need a TabularDataset and not a dataframe, therefore the current df will be converterd 
#into a TabularDataset:

#Convert the dataframe into a csv
local_path = 'prepared.csv'

#Save it locally
dataset.to_csv(local_path,index=None)

#Generate the a datastore object which is the the default datastore
datastore = ws.get_default_datastore()

In [17]:
#Upload the dataframe which was previosly converted into a csv
datastore.upload(src_dir='.', target_path='data')

Uploading an estimated of 12 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 12
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 12
Uploading ./automl6.ipynb
Uploaded ./automl6.ipynb, 3 files out of an estimated total of 12
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 4 files out of an estimated total of 12
Uploading ./prepared.csv
Uploaded ./prepared.csv, 5 files out of an estimated total of 12
Uploading ./pre_preprocess.py
Uploaded ./pre_preprocess.py, 6 files out of an estimated total of 12
Uploading ./automl6.ipynb.amltmp
Uploaded ./automl6.ipynb.amltmp, 7 files out of an estimated total of 12
Uploading ./score.py
Uploaded ./score.py, 8 files out of an estimated total of 12
Uploading ./train.py
Uploaded ./train.py, 9 files out of an estimated total of 12
Uploading ./.ipynb_aml_checkpoints/automl6-checkpoint2021-1-8-16-20-12.ipynb
Uploaded ./.ipynb_aml_checkpoi

$AZUREML_DATAREFERENCE_b07a40ffff4d4a4f8031b2b7150e30bb

In [18]:
#For the sake of checking; check the path
datastore.path()

$AZUREML_DATAREFERENCE_workspaceblobstore

In [19]:
#Now the uploaded file will be transformed into a Tabular dataset and store in a varible named 'training_dataset'
training_dataset = Dataset.Tabular.from_delimited_files(path= [(datastore,('data/prepared.csv'))])

In [20]:
#let's visualize the data:
training_dataset.to_pandas_dataframe().head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
0,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
1,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
2,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
3,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
4,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


## AutoML Configuration

Below we will chose the automl settings and cofiguration

In [22]:
#Create the automl settings which will be used as argurments in the automl config
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

#Create the automl_config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=training_dataset,
                             label_column_name="survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
#Submitt the experiment

automl_run = experiment.submit(automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on notebook138164 with default configuration
Running on remote compute: notebook138164
Parent Run ID: AutoML_4650437b-1121-4754-b1dc-0f194ec52deb

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs we

In [24]:
#Additional Run Details
from azureml.widgets import RunDetails

RunDetails(automl_run).show()

# wait for completion
automl_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_4650437b-1121-4754-b1dc-0f194ec52deb',
 'target': 'notebook138164',
 'status': 'Completed',
 'startTimeUtc': '2021-02-08T16:28:21.479984Z',
 'endTimeUtc': '2021-02-08T16:47:58.055551Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook138164',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment-1","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-138164","workspace_name":"quick-starts-ws-138164","region":"southcentralus","compute_target":"notebook138164","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"n

In [25]:
#Get generic outputs from the automl_run
automl_run.get_output()


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


(Run(Experiment: automl-experiment-1,
 Id: AutoML_4650437b-1121-4754-b1dc-0f194ec52deb_46,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=1.7708333333333335,
                                                                                                scale_pos_weight=1,
 

In [26]:
#Get the best model outputs
best_automl_run, best_model = automl_run.get_output()


# Retrieve the best automl run model
print('Best AutoML run: ', best_automl_run)
print('Best AutoML model :', best_model)

# get best model and display properties
model_name = best_automl_run.properties['model_name']
print('Best_model name: ', model_name)

# display all the properties of the best model
best_automl_run.get_properties()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Best AutoML run:  Run(Experiment: automl-experiment-1,
Id: AutoML_4650437b-1121-4754-b1dc-0f194ec52deb_46,
Type: azureml.scriptrun,
Status: Completed)
Best AutoML model : Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_lambda=1.7708333333333335,
                                                                                               

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl-experiment-1\',\'compute_target\':\'notebook138164\',\'subscription_id\':\'1b944a9b-fdae-4f97-aeb1-b7eea0beac53\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_4650437b-1121-4754-b1dc-0f194ec52deb_46","experiment_name":"automl-experiment-1","workspace_name":"quick-starts-ws-138164","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group_name":"aml-quickstarts-138164"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '46',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.Co

In [27]:
#Get the best model id
print(best_automl_run.id)

AutoML_4650437b-1121-4754-b1dc-0f194ec52deb_46


In [28]:
#Save the best AutoML model

import joblib

joblib.dump(best_model, 'best_automl_model.pkl')

['best_automl_model.pkl']

In [29]:
# register best AutoML model for future deployment

from azureml.core.model import Model
description = 'AutoML Model trained on the titanic dataset'
tags = {'area': 'data science beginners', 'type': 'classification'}

automl_model = Model.register(workspace =ws,
                              model_name = 'best-titanicMLmodel',
                              model_path = 'best_automl_model.pkl',
                             description = description, tags = tags)

print('AutoML RunID: ', automl_run.id, sep='\t')

Registering model best-titanicMLmodel
AutoML RunID: 	AutoML_4650437b-1121-4754-b1dc-0f194ec52deb


In [ ]:
#Prepare deploying of the model as a web service
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies


In [53]:
#Setup Environment
env = Environment.get(workspace=ws, name='AzureML-AutoML')

In [54]:
#Chekc environment dependencies
print("packages", env.python.conda_dependencies.serialize_to_string())

packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.21.0.post1
  - azureml-pipeline-core==1.21.0
  - azureml-telemetry==1.21.0
  - azureml-defaults==1.21.0
  - azureml-interpret==1.21.0
  - azureml-automl-core==1.21.0
  - azureml-automl-runtime==1.21.0
  - azureml-train-automl-client==1.21.0
  - azureml-train-automl-runtime==1.21.0.post1
  - azureml-dataset-runtime==1.21.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_7ade26eb614f97df8030bc480da59236



In [55]:
#setup the inference and aci config
inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True, auth_enabled=True)


In [56]:
#Deploy
service_name = 'my-ml-service'

model = Model(ws,name='best-titanicMLmodel')
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [57]:
service.get_logs()

'/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-02-08T17:48:34,348471553+00:00 - iot-server/run \n2021-02-08T17:48:34,349282809+00:00 - rsyslog/run \n2021-02-08T17:48:34,350975325+00:00 - nginx/run \n2021-02-08T17:48:34,350966925+00:00 - gunicorn/run 

In [58]:
# print service state
print(service.state)
# print scoring URI
print('scoring URI: ' + service.scoring_uri)
# print Swagger URI
print('Swagger URI: ' + service.swagger_uri)
# retrieve authentication keys
primary, secondary = service.get_keys()
# print primary authenticaton key
print('Primary Authentication Key: ' + primary)

Healthy
scoring URI: http://87897773-cb15-40d5-ba0d-ba8285d8f467.southcentralus.azurecontainer.io/score
Swagger URI: http://87897773-cb15-40d5-ba0d-ba8285d8f467.southcentralus.azurecontainer.io/swagger.json
Primary Authentication Key: iBX2glUB3xcahOndX5AW62WoVbRiDcIZ


In [59]:
#Store the uri's in variables:

scoring_uri = 'http://87897773-cb15-40d5-ba0d-ba8285d8f467.southcentralus.azurecontainer.io/score'

key = 'iBX2glUB3xcahOndX5AW62WoVbRiDcIZ'

In [66]:
#let's test requests:
import json
import requests

scoring_uri = scoring_uri
key = key

headers = {'Content-Type':'application/json'}
headers['Authorization'] = f'Bearer {key}'


test_data = json.dumps({'data':[{
    'pclass': 0.8419164182590155,
    'age': -0.34907541344456255,
    'sibsp': -0.47908676070718687,
    'parch': -0.444999501816175,
    'fare': -0.4902404567566683,
    'age_NA': -0.5014319838391105,
    'fare_NA': -0.027650063180466557,
    'sex_male': 0.743496915331831,
    'cabin_Missing': 0.5393765119990418,
    'cabin_Rare': -0.42592011250734235,
    'embarked_Q': -0.32204029159373954,
    'embarked_Rare': -0.03911805059269843,
    'embarked_S': 0.6573935670276714,
    'title_Mr': 0.8525918887485938,
    'title_Mrs': -0.42592011250734235,
    'title_Rare': -0.27494677157229536
    }
    ]
        })

test_data2 = json.dumps({'data':[{
    'pclass': -15460978645168200,
    'age': 0.8912042887450313,
    'sibsp': -0.47908676070718687,
    'parch': -0.444999501816175,
    'fare': 19569900306355100,
    'age_NA': -0.5014319838391105,
    'fare_NA': -0.027650063180466557,
    'sex_male': -13449954927569300,
    'cabin_Missing': -18539924853119600,
    'cabin_Rare': 23478581326275300,
    'embarked_Q': -0.32204029159373954,
    'embarked_Rare': -0.03911805059269843,
    'embarked_S': -15211587854766800,
    'title_Mr': -11728941046668400,
    'title_Mrs': -0.42592011250734235,
    'title_Rare': -0.27494677157229536

    }
    ]
        })


response1 = requests.post(scoring_uri, data=test_data, headers=headers)
response2 = requests.post(scoring_uri, data=test_data2, headers=headers)

print("Classification Prediction:",response1.text)
print("Classification Prediction:",response2.text)

Classification Prediction: [0]
Classification Prediction: [1]


In [64]:
#get the environment Details and stored them into a file:
f = open("env.yml", "w")
f.write(env.python.conda_dependencies.serialize_to_string())
f.close()

print("packages", env.python.conda_dependencies.serialize_to_string())

packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.21.0.post1
  - azureml-pipeline-core==1.21.0
  - azureml-telemetry==1.21.0
  - azureml-defaults==1.21.0
  - azureml-interpret==1.21.0
  - azureml-automl-core==1.21.0
  - azureml-automl-runtime==1.21.0
  - azureml-train-automl-client==1.21.0
  - azureml-train-automl-runtime==1.21.0.post1
  - azureml-dataset-runtime==1.21.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_7ade26eb614f97df8030bc480da59236



In [ ]:
#Delete Service:
service.delete()